# 1. Import Libraries and Tools and Create Helper Functions

In [1]:
! pip install chembl_webresource_client xgboost shap rdkit joblib matplotlib

In [2]:
from chembl_webresource_client.new_client import new_client
import pandas as pd
import numpy as np
from typing import List, Optional, Tuple, Dict
from tqdm.auto import tqdm
from rdkit import Chem, DataStructs
from rdkit.Chem import Descriptors, rdMolDescriptors, AllChem, MACCSkeys, DataStructs
from rdkit.Chem.Descriptors import MolLogP

In [3]:
#=============================================================
# Helper Functions
#=============================================================

# Canonicalize smiles safely
def canonical_smiles(smiles):
    try:
        m = Chem.MolFromSmiles(smiles)
        if m is None:
            return None
        return Chem.MolToSmiles(m, isomericSmiles=True)
    except Exception:
        return None

# Convert standard value + unit to nM
def to_nM(val, unit):
    if pd.isna(val) or pd.isna(unit):
        return np.nan
    try:
        v = float(val)
    except:
        return np.nan
    u = unit.strip().lower()
    if 'nm' in u:
        return v
    if 'µm' in u or 'um' in u or 'μm' in u:
        return v * 1e3
    if u == 'm' or u == 'mol/l' or 'mol' in u:
        # assume M
        return v * 1e9
    # fallback: try to parse prefixes
    if u == 'mm':
        return v * 1e6
    return np.nan

# Convert IC50_nM TO pIC50
def to_pIC50(ic50_nM):
    # pIC50 = -log10(IC50 [M]) = 9 - log10(IC50 [nM])
    if ic50_nM <= 0 or pd.isna(ic50_nM):
        return np.nan
    return 9.0 - np.log10(ic50_nM)

In [4]:
def find_hiv_targets(client, search_term="protease"):
    """
    Search ChEMBL for HIV related targets.

    Parameters
    ----------
    client : chembl_webresource_client.new_client
        A ChEMBL new_client instance.
    search_term : str, optional
        Broad search term (default: "protease").

    Returns
    -------
    candidates : pd.DataFrame
        Filtered DataFrame of likely HIV/protease targets
        with chembl_id, pref_name, organism, and component_text.
    """

    tc = client.target
    hits = tc.search(search_term)

    rows = []
    for h in hits:
        tid = h.get('target_chembl_id')
        pname = h.get('pref_name') or ''
        org = h.get('organism') or ''

        comp_text = ''
        for c in h.get('target_components') or []:
            comp_text += ' ' + (c.get('component_description') or '')
            for s in c.get('target_component_synonyms') or []:
                comp_text += ' ' + (s.get('component_synonym') or '')

        rows.append({
            'chembl_id': tid,
            'pref_name': pname,
            'organism': org,
            'component_text': comp_text
        })

    targets_df = pd.DataFrame(rows).drop_duplicates().reset_index(drop=True)

    # filter for HIV/protease-related targets
    mask = (
        targets_df['component_text'].str.lower().str.contains('hiv|human immunodeficiency', na=False) |
        targets_df['pref_name'].str.lower().str.contains(f'hiv|{search_term}', na=False) |
        targets_df['organism'].str.lower().str.contains('hiv|human immunodeficiency', na=False)
    )

    candidates = targets_df[mask].reset_index(drop=True)

    print(f"Found {len(candidates)} candidate target(s) mentioning 'HIV' or '{search_term}e'.")
    if len(candidates) == 0:
        print(f"No obvious HIV hits found. Try search terms like 'HIV-1 {search_term}' or by known accession.")
    return candidates[['chembl_id', 'pref_name', 'organism', 'component_text']]

# 2. Curate Data for HIV-1 protease inhibitors (PIs)

In [5]:
# ==============================================================================
# Search targets and show likely HIV protease candidates
# ==============================================================================
candidates = find_hiv_targets(new_client, search_term="protease")
display(candidates)

Found 109 candidate target(s) mentioning 'HIV' or 'proteasee'.


,chembl_id,pref_name,organism,component_text
0,CHEMBL2857,Human rhinovirus A protease,Human rhinovirus sp.,Genome polyprotein Genome polyprotein protease
1,CHEMBL2366517,Protease,Human immunodeficiency virus 1,Protease protease Protease
2,CHEMBL3638323,HIV protease,Human immunodeficiency virus,HIV-1 protease HIV-1 HIV-1 protease protease
3,CHEMBL4296312,HIV-1 protease,Human immunodeficiency virus,HIV-1 protease HIV-1 HIV-1 protease protease
4,CHEMBL5297,Protease,Human T-lymphotropic virus 1,Protease Protease
...,...,...,...,...
104,CHEMBL3638326,Gag-Pol polyprotein,Human immunodeficiency virus type 1 group M su...,Gag-Pol polyprotein 2.7.7.- 2.7.7.49 2.7.7.7 ...
105,CHEMBL3638331,Gag-Pol polyprotein,Human immunodeficiency virus type 1 group M su...,Gag-Pol polyprotein 2.7.7.- 2.7.7.49 2.7.7.7 ...
106,CHEMBL3638352,Gag-Pol polyprotein,Human immunodeficiency virus type 1 group M su...,Gag-Pol polyprotein 2.7.7.- 2.7.7.49 2.7.7.7 ...
107,CHEMBL3638360,Gag-Pol polyprotein,Human immunodeficiency virus type 1 group M su...,Gag-Pol polyprotein 2.7.7.- 2.7.7.49 2.7.7.7 ...


In [6]:
# ================================================
# Fetch IC50 activities for chosen target(s)
# ================================================
activity_client = new_client.activity

# There are only two target HIV-1 proteases
target_ids = ['CHEMBL2366517', 'CHEMBL3638323']

all_acts = []
for tid in target_ids:
    print("Downloading activities for", tid)
    # fetch activities filtered by standard_type=IC50 (iterable)
    acts_iter = activity_client.filter(target_chembl_id=tid, standard_type='IC50')
    acts = list(acts_iter)  # convert to list
    print(" -> returned", len(acts), "rows")
    for a in acts:
        a['_queried_target'] = tid
    all_acts.extend(acts)

print("Total activity rows collected:", len(all_acts))
acts_df = pd.DataFrame(all_acts)
# quick glance
display(acts_df.head())


 -> returned 885 rows
 -> returned 44 rows
Total activity rows collected: 929


,action_type,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,...,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value,_queried_target
0,None,None,1985918,[],CHEMBL899796,Inhibition of HIV1 recombinant protease,B,None,None,BAO_0000190,...,Protease,11676,None,None,IC50,nM,UO_0000065,None,21.1,CHEMBL2366517
1,None,None,1985919,[],CHEMBL899796,Inhibition of HIV1 recombinant protease,B,None,None,BAO_0000190,...,Protease,11676,None,None,IC50,nM,UO_0000065,None,153.9,CHEMBL2366517
2,None,None,1985920,[],CHEMBL899796,Inhibition of HIV1 recombinant protease,B,None,None,BAO_0000190,...,Protease,11676,None,None,IC50,nM,UO_0000065,None,37.3,CHEMBL2366517
3,None,None,1985921,[],CHEMBL899796,Inhibition of HIV1 recombinant protease,B,None,None,BAO_0000190,...,Protease,11676,None,None,IC50,nM,UO_0000065,None,2.5,CHEMBL2366517
4,None,None,1985922,[],CHEMBL899796,Inhibition of HIV1 recombinant protease,B,None,None,BAO_0000190,...,Protease,11676,None,None,IC50,nM,UO_0000065,None,10000.0,CHEMBL2366517


In [7]:
acts_df.columns

Index(['action_type', 'activity_comment', 'activity_id', 'activity_properties',
       'assay_chembl_id', 'assay_description', 'assay_type',
       'assay_variant_accession', 'assay_variant_mutation', 'bao_endpoint',
       'bao_format', 'bao_label', 'canonical_smiles', 'data_validity_comment',
       'data_validity_description', 'document_chembl_id', 'document_journal',
       'document_year', 'ligand_efficiency', 'molecule_chembl_id',
       'molecule_pref_name', 'parent_molecule_chembl_id', 'pchembl_value',
       'potential_duplicate', 'qudt_units', 'record_id', 'relation', 'src_id',
       'standard_flag', 'standard_relation', 'standard_text_value',
       'standard_type', 'standard_units', 'standard_upper_value',
       'standard_value', 'target_chembl_id', 'target_organism',
       'target_pref_name', 'target_tax_id', 'text_value', 'toid', 'type',
       'units', 'uo_units', 'upper_value', 'value', '_queried_target'],
      dtype='object')

In [8]:
cols = ['molecule_chembl_id', 'canonical_smiles', 'standard_type', 'standard_value', 'standard_units', 'pchembl_value', 'assay_chembl_id', 'assay_description', 'target_chembl_id', 'target_pref_name', 'target_organism', 'standard_flag', 'potential_duplicate', 'data_validity_comment', 'data_validity_description']
pi_df = acts_df[cols]
pi_df.head()

,molecule_chembl_id,canonical_smiles,standard_type,standard_value,standard_units,pchembl_value,assay_chembl_id,assay_description,target_chembl_id,target_pref_name,target_organism,standard_flag,potential_duplicate,data_validity_comment,data_validity_description
0,CHEMBL1627209,O=C(N[C@@H]1c2ccccc2C[C@@H]1O)[C@@H](Cc1ccccc1...,IC50,21.1,nM,7.68,CHEMBL899796,Inhibition of HIV1 recombinant protease,CHEMBL2366517,Protease,Human immunodeficiency virus 1,1,0,None,None
1,CHEMBL1627287,N=[S+]([O-])([C@H](Cc1ccccc1)C(=O)N[C@H]1c2ccc...,IC50,153.9,nM,6.81,CHEMBL899796,Inhibition of HIV1 recombinant protease,CHEMBL2366517,Protease,Human immunodeficiency virus 1,1,0,None,None
2,CHEMBL1627235,N=[S+]([O-])([C@H](Cc1ccccc1)C(=O)N[C@H]1c2ccc...,IC50,37.3,nM,7.43,CHEMBL899796,Inhibition of HIV1 recombinant protease,CHEMBL2366517,Protease,Human immunodeficiency virus 1,1,0,None,None
3,CHEMBL1627210,N=[S+]([O-])(C[C@H](Cc1ccccc1)C(=O)N[C@@H]1c2c...,IC50,2.5,nM,8.60,CHEMBL899796,Inhibition of HIV1 recombinant protease,CHEMBL2366517,Protease,Human immunodeficiency virus 1,1,0,None,None
4,CHEMBL396814,O=C(N[C@H]1c2ccccc2C[C@H]1O)[C@@H](Cc1ccccc1)C...,IC50,10000.0,nM,None,CHEMBL899796,Inhibition of HIV1 recombinant protease,CHEMBL2366517,Protease,Human immunodeficiency virus 1,1,0,None,None


In [9]:
pi_df['standard_units'].value_counts()

,count
standard_units,
nM,854
ug.mL-1,57


In [10]:
print(pi_df['data_validity_comment'].value_counts())
print(pi_df['potential_duplicate'].value_counts())
print(pi_df['data_validity_description'].value_counts())

data_validity_comment
Outside typical range    53
Name: count, dtype: int64
potential_duplicate
0    905
1     24
Name: count, dtype: int64
data_validity_description
Values for this activity type are unusually large/small, so may not be accurate    53
Name: count, dtype: int64


In [11]:
# keep rows where potential_duplicate is not True
pi_df = pi_df[pi_df['potential_duplicate'] != True]

# keep rows where data_validity_comment is null/NaN
pi_df = pi_df[pi_df['data_validity_comment'].isna()]

pi_df.shape

(852, 15)

In [12]:
print(pi_df['data_validity_comment'].value_counts())
print(pi_df['potential_duplicate'].value_counts())
print(pi_df['data_validity_description'].value_counts())

Series([], Name: count, dtype: int64)
potential_duplicate
0    852
Name: count, dtype: int64
Series([], Name: count, dtype: int64)


In [13]:
pi_df.to_parquet('pi_df.parquet', index=False)

# 3. Standardize IC50 and Deduplicate the data

In [14]:
# normalize unit strings
def norm_unit(u):
    if pd.isna(u):
        return u
    if u == 'ug.mL-1':
        return 'ug/mL'
    return u


# compute mol weight for each unique canonical_smiles
def molwt_from_smiles(smi):
    if pd.isna(smi):
        return np.nan
    try:
        m = Chem.MolFromSmiles(smi)
        if m is None:
            return np.nan
        return Descriptors.MolWt(m)  # g/mol
    except:
        return np.nan


# function to convert a row to nM
def value_to_nM(row):
    val = row['standard_value']
    unit = row['standard_units_norm']
    # ensure numeric
    try:
        v = float(val)
    except:
        return np.nan
    if pd.isna(unit):
        return np.nan

    if unit == 'nM':
        return v

    elif unit == 'ug/mL':
        mw = row.get('mol_wt_g_per_mol', np.nan)
        if pd.isna(mw) or mw == 0:
            return np.nan
        # nM = v * 1e6 / MW
        return float(v) * 1e6 / float(mw)

    # add more conversions if you see additional units
    return np.nan


# compute pIC50
def to_pIC50(nM):
    if pd.isna(nM) or nM <= 0:
        return np.nan
    return 9.0 - np.log10(nM)

In [15]:
pi_df = pd.read_parquet('pi_df.parquet')

In [16]:
print(pi_df.shape)

(852, 15)


In [17]:
pi_df['standard_units_norm'] = pi_df['standard_units'].astype(str).apply(norm_unit)

# compute MW per unique smiles and map back
unique_smiles = pi_df['canonical_smiles'].dropna().unique().tolist()
mw_map = {}
for s in unique_smiles:
    mw_map[s] = molwt_from_smiles(s)

# add MW column
pi_df['mol_wt_g_per_mol'] = pi_df['canonical_smiles'].map(mw_map)

pi_df['IC50_nM'] = pi_df.apply(value_to_nM, axis=1)

pi_df['pIC50'] = pi_df['IC50_nM'].apply(to_pIC50)

# Quick diagnostics: show counts by unit and how many converted sucessfully
print("Unit value counts:")
print(pi_df['standard_units_norm'].value_counts(dropna=False))
print("\nConversion success counts (IC50_nM not null):")
print(pi_df.groupby('standard_units_norm')['IC50_nM'].apply(lambda s: s.notna().sum()))

# Flag rows where conversion failed (mass units but no MW)
mask_mass_units = pi_df['standard_units_norm'].str.contains('ug|mg', na=False)
failed = pi_df[mask_mass_units & pi_df['IC50_nM'].isna()]
print("\nRows with mass units that could not be converted (need manual inspection):", len(failed))
display(failed.head(10))

Unit value counts:
standard_units_norm
nM       788
ug/mL     46
None      18
Name: count, dtype: int64

Conversion success counts (IC50_nM not null):
standard_units_norm
None       0
nM       788
ug/mL     46
Name: IC50_nM, dtype: int64

Rows with mass units that could not be converted (need manual inspection): 0


,molecule_chembl_id,canonical_smiles,standard_type,standard_value,standard_units,pchembl_value,assay_chembl_id,assay_description,target_chembl_id,target_pref_name,target_organism,standard_flag,potential_duplicate,data_validity_comment,data_validity_description,standard_units_norm,mol_wt_g_per_mol,IC50_nM,pIC50


In [18]:
pi_df['standard_units_norm'].value_counts()

,count
standard_units_norm,
nM,788
ug/mL,46
None,18


In [19]:
# Drop rows where standard_units_norm is None (or missing)
pi_df_clean = pi_df[pi_df['standard_units_norm']!='None']
pi_df_clean['standard_units_norm'].value_counts()

,count
standard_units_norm,
nM,788
ug/mL,46


In [20]:
pi_df.columns

Index(['molecule_chembl_id', 'canonical_smiles', 'standard_type',
       'standard_value', 'standard_units', 'pchembl_value', 'assay_chembl_id',
       'assay_description', 'target_chembl_id', 'target_pref_name',
       'target_organism', 'standard_flag', 'potential_duplicate',
       'data_validity_comment', 'data_validity_description',
       'standard_units_norm', 'mol_wt_g_per_mol', 'IC50_nM', 'pIC50'],
      dtype='object')

In [21]:
pi_df_clean.drop(columns=['standard_units', 'standard_flag', 'potential_duplicate',
                          'data_validity_comment', 'data_validity_description', 'mol_wt_g_per_mol'],
                 inplace=True)

print(pi_df.shape)
print(pi_df_clean.shape)

(852, 19)
(834, 13)


/tmp/ipython-input-307214963.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pi_df_clean.drop(columns=['standard_units', 'standard_flag', 'potential_duplicate',


In [22]:
pi_df_clean.to_parquet('pi_df_clean.parquet', index=False)

# 4. Compute Descriptors and Fingerprints

In [23]:
# ------------------------------------------------------------------------------
# Utilities: SMILES -> Mol
# ------------------------------------------------------------------------------
def mol_from_smiles_safe(smi: str) -> Optional[Chem.Mol]:
    """Return RDKit Mol for a SMILES string or None if parsing fails."""
    if smi is None or (isinstance(smi, float) and np.isnan(smi)):
        return None
    try:
        m = Chem.MolFromSmiles(str(smi))
        return m
    except Exception:
        return None

def add_rdkit_mols(df: pd.DataFrame, smiles_col: str = 'canonical_smiles', mol_col: str = 'rdkit_mol',
                   show_progress: bool = False) -> pd.DataFrame:
    """Add a column with RDKit Mol objects parsed from SMILES. Returns new DataFrame (copy)."""
    df = df.copy()
    seq = df[smiles_col].tolist()
    mols = []
    if show_progress:
        iterable = tqdm(seq, desc='Parsing SMILES')
    else:
        iterable = seq
    for s in iterable:
        mols.append(mol_from_smiles_safe(s))
    df[mol_col] = mols
    return df

# ------------------------------------------------------------------------------
# Descriptor calculation
# ------------------------------------------------------------------------------
DEFAULT_DESC_LIST = [
    'MolWt', 'MolLogP', 'MolMR', 'TPSA', 'NumHDonors', 'NumHAcceptors',
    'NumRotatableBonds', 'NumAromaticRings', 'HeavyAtomCount', 'FractionCSP3'
]

def compute_descriptors_for_mol(m: Chem.Mol) -> Dict[str, float]:
    """Compute a set of basic 2D descriptors for a single RDKit Mol."""
    if m is None:
        return {k: np.nan for k in DEFAULT_DESC_LIST}
    try:
        molwt = Descriptors.MolWt(m)
        logp, mr = rdMolDescriptors.CalcCrippenDescriptors(m)
        tpsa = rdMolDescriptors.CalcTPSA(m)
        hbd = rdMolDescriptors.CalcNumHBD(m)
        hba = rdMolDescriptors.CalcNumHBA(m)
        rot = rdMolDescriptors.CalcNumRotatableBonds(m)
        arom = rdMolDescriptors.CalcNumAromaticRings(m)
        hac = Descriptors.HeavyAtomCount(m)
        fsp3 = rdMolDescriptors.CalcFractionCSP3(m)
        return {
            'MolWt': float(molwt),
            'MolLogP': float(logp),
            'MolMR': float(mr),
            'TPSA': float(tpsa),
            'NumHDonors': int(hbd),
            'NumHAcceptors': int(hba),
            'NumRotatableBonds': int(rot),
            'NumAromaticRings': int(arom),
            'HeavyAtomCount': int(hac),
            'FractionCSP3': float(fsp3)
        }
    except Exception:
        return {k: np.nan for k in DEFAULT_DESC_LIST}

def compute_descriptors_df(df: pd.DataFrame, mol_col: str = 'rdkit_mol',
                           desc_names: Optional[List[str]] = None,
                           show_progress: bool = False) -> pd.DataFrame:
    """Compute descriptors for all molecules in df[mol_col] and return a DataFrame aligned with df index."""
    if desc_names is None:
        desc_names = DEFAULT_DESC_LIST
    rows = []
    seq = df[mol_col].tolist()
    if show_progress:
        iterable = tqdm(seq, desc='Computing descriptors')
    else:
        iterable = seq
    for m in iterable:
        rows.append(compute_descriptors_for_mol(m))
    desc_df = pd.DataFrame(rows, index=df.index)[desc_names]
    return desc_df

# ------------------------------------------------------------------------------
# Fingerprint calculations
# ------------------------------------------------------------------------------
def morgan_fp_array(m: Chem.Mol, radius: int = 2, nBits: int = 2048) -> np.ndarray:
    """Return numpy array (0/1 int) of Morgan fingerprint for a single RDKit Mol."""
    if m is None:
        return np.zeros((nBits,), dtype=np.uint8)
    try:
        bitvect = AllChem.GetMorganFingerprintAsBitVect(m, radius, nBits=nBits)
        arr = np.zeros((nBits,), dtype=np.uint8)
        DataStructs.ConvertToNumpyArray(bitvect, arr)
        return arr
    except Exception:
        return np.zeros((nBits,), dtype=np.uint8)

def compute_fingerprints_arrays(df: pd.DataFrame, mol_col: str = 'rdkit_mol',
                                morgan_radius: int = 2, morgan_nbits: int = 512,
                                show_progress: bool = False
                               ) -> Tuple[np.ndarray, Optional[np.ndarray]]:
    """Compute Morgan fingerprint array (n_samples, nBits). Optionally compute MACCS (n_samples, 167).
       Returns morgan_array."""
    mols = df[mol_col].tolist()
    if show_progress:
        mols_iter = tqdm(mols, desc='Computing fingerprints')
    else:
        mols_iter = mols
    morgan_list = [morgan_fp_array(m, radius=morgan_radius, nBits=morgan_nbits) for m in mols_iter]
    morgan_arr = np.vstack(morgan_list).astype(np.uint8)
    return morgan_arr

# ------------------------------------------------------------------------------
# Helpers to convert fp arrays -> DataFrame with bit columns
# ------------------------------------------------------------------------------
def fp_array_to_df(fp_arr: np.ndarray, prefix: str = 'FP') -> pd.DataFrame:
    """Convert 2D numpy fingerprint array (n_samples, nBits) to DataFrame with columns FP_0..FP_{nBits-1}."""
    nBits = fp_arr.shape[1]
    cols = [f'{prefix}_{i}' for i in range(nBits)]
    return pd.DataFrame(fp_arr, columns=cols, index=range(fp_arr.shape[0]))

# ------------------------------------------------------------------------------
# Build feature DataFrame
# ------------------------------------------------------------------------------
def build_feature_dataframe(df: pd.DataFrame,
                            smiles_col: str = 'canonical_smiles',
                            mol_col: str = 'rdkit_mol',
                            desc_names: Optional[List[str]] = None,
                            morgan_radius: int = 2,
                            morgan_nbits: int = 512,
                            show_progress: bool = False
                           ) -> Tuple[pd.DataFrame, np.ndarray, Optional[np.ndarray]]:
    """Given a DataFrame with SMILES, return:
       - feature_df: descriptors + fingerprint bit columns (pandas DataFrame)
       - morgan_array: (n_samples, nBits) numpy array
    """
    # 1) ensure rdkit mols exist
    if mol_col not in df.columns:
        df = add_rdkit_mols(df, smiles_col=smiles_col, mol_col=mol_col, show_progress=show_progress)
    # 2) descriptors
    desc_df = compute_descriptors_df(df, mol_col=mol_col, desc_names=desc_names, show_progress=show_progress)
    # 3) fingerprints arrays
    morgan_arr= compute_fingerprints_arrays(df, mol_col=mol_col,
                                            morgan_radius=morgan_radius, morgan_nbits=morgan_nbits,
                                            show_progress=show_progress)
    # 4) convert fp arrays to DataFrame
    morgan_df = fp_array_to_df(morgan_arr, prefix=f'Morgan_{morgan_nbits}')
    feature_df = pd.concat([desc_df.reset_index(drop=True), morgan_df.reset_index(drop=True)], axis=1)
    return feature_df, morgan_arr

# ------------------------------------------------------------------------------
# Save/load helpers
# ------------------------------------------------------------------------------
def save_feature_arrays(prefix: str,
                        feature_df: pd.DataFrame,
                        morgan_arr: np.ndarray,
                        ) -> Dict[str,str]:
    """Save feature DataFrame to parquet and fingerprint arrays as .npy files.
       Returns dict of saved paths.
    """
    feature_path = f'{prefix}_features.parquet'
    morgan_path = f'{prefix}_morgan.npy'
    pd.DataFrame(feature_df).to_parquet(feature_path, index=False)
    np.save(morgan_path, morgan_arr)
    saved = {'features_parquet': feature_path, 'morgan_npy': morgan_path}
    return saved

def load_feature_arrays(prefix: str) -> Tuple[pd.DataFrame, np.ndarray, Optional[np.ndarray]]:
    """Load previously saved files (expects same naming used in save_feature_arrays)."""
    feature_path = f'{prefix}_features.parquet'
    morgan_path = f'{prefix}_morgan.npy'
    feature_df = pd.read_parquet(feature_path)
    morgan_arr = np.load(morgan_path)
    return feature_df, morgan_arr

# ------------------------------------------------------------------------------
# Example quick diagnostics
# ------------------------------------------------------------------------------
def fingerprint_density_stats(morgan_arr: np.ndarray) -> Dict[str, float]:
    """Return simple stats: mean bit density per fingerprint, fraction of bits with >0 frequency."""
    if morgan_arr is None:
        return {}
    # average number of bits set per sample
    bits_per_sample = morgan_arr.sum(axis=1)
    mean_bits = float(bits_per_sample.mean())
    # fraction of fingerprint bits that are ever on
    bits_active = (morgan_arr.sum(axis=0) > 0).mean()
    return {'mean_bits_set_per_sample': mean_bits, 'fraction_bits_active': float(bits_active)}

# ------------------------------------------------------------------------------
# Convenience: aggregate duplicates by SMILES (median target)
# ------------------------------------------------------------------------------
def aggregate_by_smiles(df: pd.DataFrame, smiles_col: str = 'canonical_smiles',
                        value_col: str = 'pIC50') -> pd.DataFrame:
    """Aggregate rows by canonical SMILES, returning DataFrame with unique smiles and median value_col.
       Keeps the first molecule_chembl_id if present.
    """
    df = df.copy()
    df['can_smiles'] = df[smiles_col]
    agg = df.groupby('can_smiles').agg({
        value_col: 'median',
        'molecule_chembl_id': lambda x: x.dropna().astype(str).unique().tolist()
    }).rename(columns={value_col: f'median_{value_col}'}).reset_index()
    return agg

In [24]:
pi_df = pd.read_parquet('pi_df_clean.parquet')

In [25]:
protease = pi_df.copy()

# Add RDKit mols (safe)
protease = add_rdkit_mols(protease, smiles_col='canonical_smiles', mol_col='rdkit_mol', show_progress=True)

# Drop rows where SMILES failed
protease = protease[protease['rdkit_mol'].notna()].reset_index(drop=True)

# Build features
feature_prot, morgan_arr = build_feature_dataframe(protease, smiles_col='canonical_smiles',
                                                morgan_radius=2, morgan_nbits=512,
                                                show_progress=True)

# attach pIC50 to features for modelling
feature_prot['pIC50'] = protease['pIC50'].values

# Save
paths = save_feature_arrays('protease_qsar', feature_prot, morgan_arr)
print(paths)

Parsing SMILES:   0%|          | 0/834 [00:00<?, ?it/s]

Computing descriptors:   0%|          | 0/834 [00:00<?, ?it/s]

Computing fingerprints:   0%|          | 0/834 [00:00<?, ?it/s]

[18:05:55] DEPRECATION WARNING: please use MorganGenerator
[18:05:55] DEPRECATION WARNING: please use MorganGenerator
[18:05:55] DEPRECATION WARNING: please use MorganGenerator
[18:05:55] DEPRECATION WARNING: please use MorganGenerator
[18:05:55] DEPRECATION WARNING: please use MorganGenerator
[18:05:55] DEPRECATION WARNING: please use MorganGenerator
[18:05:55] DEPRECATION WARNING: please use MorganGenerator
[18:05:55] DEPRECATION WARNING: please use MorganGenerator
[18:05:55] DEPRECATION WARNING: please use MorganGenerator
[18:05:55] DEPRECATION WARNING: please use MorganGenerator
[18:05:55] DEPRECATION WARNING: please use MorganGenerator
[18:05:55] DEPRECATION WARNING: please use MorganGenerator
[18:05:55] DEPRECATION WARNING: please use MorganGenerator
[18:05:55] DEPRECATION WARNING: please use MorganGenerator
[18:05:55] DEPRECATION WARNING: please use MorganGenerator
[18:05:55] DEPRECATION WARNING: please use MorganGenerator
[18:05:55] DEPRECATION WARNING: please use MorganGenerat

{'features_parquet': 'protease_qsar_features.parquet', 'morgan_npy': 'protease_qsar_morgan.npy'}


# 5. QSAR Model for protease inhibitor pIC50 value

We have appended the morgan fingerprints and molecular descriptors into a single dataframe

In [26]:
! pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 4.1 MB/s eta 0:00:00


In [27]:
# Requirements: pandas, numpy, scikit-learn, xgboost, joblib, matplotlib
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler

# Models for scaled data
from sklearn.linear_model import ElasticNet, LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor

# Tree based models
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings('ignore')

In [28]:
df = pd.read_parquet('protease_qsar_features.parquet')
print("rows, cols:", df.shape)

rows, cols: (834, 523)


In [29]:
df.head()

,MolWt,MolLogP,MolMR,TPSA,NumHDonors,NumHAcceptors,NumRotatableBonds,NumAromaticRings,HeavyAtomCount,FractionCSP3,...,Morgan_512_503,Morgan_512_504,Morgan_512_505,Morgan_512_506,Morgan_512_507,Morgan_512_508,Morgan_512_509,Morgan_512_510,Morgan_512_511,pIC50
0,636.814,4.00190,179.2214,121.72,4,5,12,4,46,0.315789,...,0,0,0,0,1,0,0,0,0,7.675718
1,623.775,3.80477,172.8031,145.57,5,6,10,4,45,0.277778,...,0,0,0,0,1,0,0,0,0,6.812761
2,623.775,3.80477,172.8031,145.57,5,6,10,4,45,0.277778,...,0,0,0,0,1,0,0,0,0,7.428291
3,651.829,4.29997,181.9411,145.57,5,6,12,4,47,0.315789,...,0,0,0,0,1,0,0,0,0,8.602060
4,636.814,4.00190,179.2214,121.72,4,5,12,4,46,0.315789,...,0,0,0,0,1,0,0,0,0,5.000000


In [30]:
# Prepare X and y
y = df['pIC50']
X = df.drop(columns=['pIC50'])
print('Feature count: ', X.shape[1])

Feature count:  522


In [31]:
# ------------------------------------------------------------------------------
# Quick feature cleanup
# ------------------------------------------------------------------------------

# a. remove near-constant features

# Fit the selector
vt = VarianceThreshold(threshold=1e-6)
X_v = vt.fit_transform(X)
# Features kept
kept_features = X.columns[vt.get_support()].tolist()
# Features removed
removed_features = [col for col in X.columns if col not in kept_features]
# Create the reduced DataFrame
X = pd.DataFrame(X_v, columns=kept_features)

print(f"Features after variance filter: {len(kept_features)}")
print(f"Features removed: {len(removed_features)}")
# Optional: display or save the removed features
print("\nRemoved features (first 20):")
print(removed_features[:20])

Features after variance filter: 521
Features removed: 1

Removed features (first 20):
['Morgan_512_499']


In [32]:
# b. (optional) remove extremely collinear features (simple correlation filter)

corr_thresh = 0.98
corr_matrix = X.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [col for col in upper.columns if any(upper[col] > corr_thresh)]
if to_drop:
    X.drop(columns=to_drop, inplace=True)
    print("Dropped highly correlated features:", to_drop)

Dropped highly correlated features: ['MolMR', 'HeavyAtomCount']


In [33]:
X.shape

(834, 519)

In [34]:
# ------------------------------------------------------------------------------
# Define descriptor columns to scale and fingerprint columns to pass through
# ------------------------------------------------------------------------------
descriptors = ['MolWt', 'MolLogP', 'TPSA', 'NumHDonors', 'NumHAcceptors',
    'NumRotatableBonds', 'NumAromaticRings', 'FractionCSP3']
fingerprints = [c for c in X.columns if c not in descriptors]

# ColumnTransformer: scale descriptors, pass fingerprints through
preprocessor = ColumnTransformer([
    ("desc", StandardScaler(), descriptors),
    ("fp", "passthrough", fingerprints)
])

In [35]:
# ---------------------------
# k-fold OOF trainer
# ---------------------------
def kfold_train_predict(model, X_df, y_arr, n_splits=5, random_state=42):
    """
    model: estimator or pipeline (should accept fit/predict)
    X_df: pandas DataFrame (full dataset)
    y_arr: numpy array or Series
    returns: oof_preds (np.array same length as X_df), mean_r2, mean_rmse
    """
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    oof_preds = np.zeros(len(X_df))
    fold_scores = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(X_df, y_arr), 1):
        X_train, X_val = X_df.iloc[train_idx], X_df.iloc[val_idx]
        y_train, y_val = y_arr[train_idx], y_arr[val_idx]

        # fit on train fold, predict on val fold
        model.fit(X_train, y_train)
        preds = model.predict(X_val)

        oof_preds[val_idx] = preds

        fold_r2 = r2_score(y_val, preds)
        fold_rmse = np.sqrt(mean_squared_error(y_val, preds))
        fold_scores.append((fold_r2, fold_rmse))
        print(f"Fold {fold}: R2 = {fold_r2:.3f}, RMSE = {fold_rmse:.3f}")

    mean_r2 = np.mean([s[0] for s in fold_scores])
    mean_rmse = np.mean([s[1] for s in fold_scores])
    print(f"\nMean CV R2 = {mean_r2:.3f}, Mean RMSE = {mean_rmse:.3f}")
    return oof_preds, mean_r2, mean_rmse

In [36]:
# ---------------------------
# Define models and pipelines
# ---------------------------
models = {}

# Linear models (need scaling) -> include preprocessor
models['Linear'] = Pipeline([("preproc", preprocessor), ("model", LinearRegression())])
models['ElasticNet'] = Pipeline([("preproc", preprocessor), ("model", ElasticNet(random_state=42, max_iter=5000))])

# SVR (needs scaling)
models['SVM'] = Pipeline([("preproc", preprocessor), ("model", SVR())])

# KNN
models['KNN'] = Pipeline([("preproc", preprocessor), ("model", KNeighborsRegressor())])

# MLP
models['MLP'] = Pipeline([("preproc", preprocessor), ("model", MLPRegressor(max_iter=2000, random_state=42))])

# Random Forest (trees don't need scaling, but pipeline is fine)
models['Random Forest'] = Pipeline([("preproc", preprocessor), ("model", RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1))])

# XGBoost
models['XGBoost'] = Pipeline([("preproc", preprocessor), ("model", XGBRegressor(n_estimators=200, random_state=42, n_jobs=-1, objective='reg:squarederror'))])

In [37]:
# ---------------------------
# Run k-fold for each model
# ---------------------------
results = {}
oof_dict = {}

for name, model in models.items():
    print(f"\n=== Training model: {name} ===")
    oof_preds, mean_r2, mean_rmse = kfold_train_predict(model, X, y, n_splits=5, random_state=42)
    results[name] = {"r2": mean_r2, "rmse": mean_rmse}
    oof_dict[name] = oof_preds

# Summary
print("\nSummary (mean CV results):")
for name, res in results.items():
    print(f"{name}: R2 = {res['r2']:.4f}, RMSE = {res['rmse']:.4f}")


=== Training model: Linear ===
Fold 1: R2 = -1.433, RMSE = 2.736
Fold 2: R2 = -0.521, RMSE = 2.214
Fold 3: R2 = -1.164, RMSE = 2.549
Fold 4: R2 = -0.456, RMSE = 2.226
Fold 5: R2 = -0.955, RMSE = 2.571

Mean CV R2 = -0.906, Mean RMSE = 2.459

=== Training model: ElasticNet ===
Fold 1: R2 = 0.121, RMSE = 1.644
Fold 2: R2 = 0.085, RMSE = 1.717
Fold 3: R2 = 0.099, RMSE = 1.644
Fold 4: R2 = 0.118, RMSE = 1.732
Fold 5: R2 = 0.116, RMSE = 1.728

Mean CV R2 = 0.108, Mean RMSE = 1.693

=== Training model: SVM ===
Fold 1: R2 = 0.785, RMSE = 0.814
Fold 2: R2 = 0.777, RMSE = 0.849
Fold 3: R2 = 0.773, RMSE = 0.826
Fold 4: R2 = 0.768, RMSE = 0.889
Fold 5: R2 = 0.744, RMSE = 0.930

Mean CV R2 = 0.769, Mean RMSE = 0.862

=== Training model: KNN ===
Fold 1: R2 = 0.780, RMSE = 0.823
Fold 2: R2 = 0.744, RMSE = 0.909
Fold 3: R2 = 0.778, RMSE = 0.817
Fold 4: R2 = 0.781, RMSE = 0.863
Fold 5: R2 = 0.747, RMSE = 0.925

Mean CV R2 = 0.766, Mean RMSE = 0.868

=== Training model: MLP ===
Fold 1: R2 = 0.760, RMS

# 6. Tune the Best Models

### Support Vector Machine (SVR),
### K-Nearest Neighbours (KNN),
### Random Forest (RF)

In [38]:
import optuna
import joblib
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, loguniform
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA

In [39]:
# PCA variant of the model
n_fp = len(fingerprints)
print(f"Using {len(descriptors)} descriptor columns and {n_fp} fingerprint columns.")

if n_fp == 0:
    raise ValueError("No fingerprint columns detected. Make sure your X contains fingerprint bit columns.")

# ---------- CV settings ----------
cv_outer = KFold(n_splits=5, shuffle=True, random_state=42)

Using 8 descriptor columns and 511 fingerprint columns.


In [40]:
# ----------------------- SVM ---------------------------------
def run_optuna_svm(n_trials=80):
    def objective(trial):
        use_pca = trial.suggest_categorical("use_pca", [True, False])
        if use_pca:
            max_comp = min(n_fp, 300)
            n_comp = trial.suggest_int("pca_n_components", 10, max(10, max_comp))
            fp_transform = Pipeline([("pca", PCA(n_components=n_comp, svd_solver="randomized", random_state=42))])
        else:
            fp_transform = "passthrough"

        preprocessor = ColumnTransformer([
            ("desc", StandardScaler(), descriptors),
            ("fp", fp_transform, fingerprints)
        ])

        # SVR hyperparameters (log-uniform sampling where appropriate)
        kernel = trial.suggest_categorical("kernel", ["rbf", "poly", "sigmoid"])
        C = trial.suggest_loguniform("C", 1e-3, 1e3)
        epsilon = trial.suggest_loguniform("epsilon", 1e-4, 1.0)
        gamma_choice = trial.suggest_categorical("gamma_choice", ["scale", "auto", "numeric"])
        if gamma_choice == "numeric":
            gamma = trial.suggest_loguniform("gamma", 1e-5, 1e1)
        else:
            gamma = gamma_choice
        # degree and coef0 for polynomial kernel
        if kernel == "poly":
            degree = trial.suggest_int("degree", 2, 5)
            coef0 = trial.suggest_float("coef0", 0.0, 1.0)
        else:
            degree = 3
            coef0 = 0.0

        svr = SVR(kernel=kernel, C=C, epsilon=epsilon, gamma=gamma, degree=degree, coef0=coef0, max_iter=100000)
        pipe = Pipeline([("preproc", preprocessor), ("svr", svr)])
        scores = cross_val_score(pipe, X, y, cv=cv_outer, scoring="r2", n_jobs=-1)
        return float(np.mean(scores))

    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(),
                                pruner=optuna.pruners.MedianPruner())
    study.optimize(objective, n_trials=n_trials, n_jobs=1, show_progress_bar=True)

    # Rebuild best pipeline and fit on full data
    best = study.best_params
    if best.get("use_pca", False):
        fp_transform = Pipeline([("pca", PCA(n_components=best["pca_n_components"], random_state=42))])
    else:
        fp_transform = "passthrough"
    preprocessor_best = ColumnTransformer([
        ("desc", StandardScaler(), descriptors),
        ("fp", fp_transform, fingerprints)
    ])
    # reconstruct SVR
    best_kernel = best.get("kernel", "rbf")
    best_gamma = best.get("gamma", best.get("gamma_choice", "scale"))
    if best_gamma == "numeric":
        best_gamma = best.get("gamma")
    best_degree = best.get("degree", 3)
    best_coef0 = best.get("coef0", 0.0)

    best_svm = SVR(
        kernel=best_kernel,
        C=best.get("C"),
        epsilon=best.get("epsilon"),
        gamma=best_gamma,
        degree=best_degree,
        coef0=best_coef0,
        max_iter=100000
    )
    best_pipe = Pipeline([("preproc", preprocessor_best), ("svr", best_svm)])
    best_pipe.fit(X, y)
    joblib.dump(best_pipe, "svm_optuna_pipeline.joblib")
    print("SVM best R2:", study.best_value)
    print("SVM best params:", study.best_params)
    return study, best_pipe

In [41]:
# ------------------ KNN -----------------------
def run_optuna_knn(n_trials=50):
    def objective(trial):
        use_pca = trial.suggest_categorical("use_pca", [True, False])
        if use_pca:
            max_comp = min(n_fp, 300)
            n_comp = trial.suggest_int("pca_n_components", 10, max(10, max_comp))
            fp_transform = Pipeline([("pca", PCA(n_components=n_comp, svd_solver="randomized", random_state=42))])
        else:
            fp_transform = "passthrough"

        preprocessor = ColumnTransformer([
            ("desc", StandardScaler(), descriptors),
            ("fp", fp_transform, fingerprints)
        ])

        n_neighbors = trial.suggest_int("n_neighbors", 1, 30)
        weights = trial.suggest_categorical("weights", ["uniform", "distance"])
        p = trial.suggest_int("p", 1, 2)  # 1 = manhattan, 2 = euclidean

        knn = KNeighborsRegressor(n_neighbors=n_neighbors, weights=weights, p=p)
        pipe = Pipeline([("preproc", preprocessor), ("knn", knn)])
        scores = cross_val_score(pipe, X, y, cv=cv_outer, scoring="r2", n_jobs=-1)
        return float(np.mean(scores))

    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(),
                                pruner=optuna.pruners.MedianPruner())
    study.optimize(objective, n_trials=n_trials, n_jobs=1, show_progress_bar=True)

    # Rebuild best pipeline and fit on full data
    best = study.best_params
    if best.get("use_pca", False):
        fp_transform = Pipeline([("pca", PCA(n_components=best["pca_n_components"], random_state=42))])
    else:
        fp_transform = "passthrough"
    preprocessor_best = ColumnTransformer([
        ("desc", StandardScaler(), descriptors),
        ("fp", fp_transform, fingerprints)
    ])
    best_knn = KNeighborsRegressor(n_neighbors=best["n_neighbors"], weights=best["weights"], p=best["p"])
    best_pipe = Pipeline([("preproc", preprocessor_best), ("knn", best_knn)])
    best_pipe.fit(X, y)
    joblib.dump(best_pipe, "knn_optuna_pipeline.joblib")
    print("KNN best R2:", study.best_value)
    print("KNN best params:", study.best_params)
    return study, best_pipe

In [42]:
# ------------------------ MLP ------------------------------
def run_optuna_mlp(n_trials=50):
    def objective(trial):
        use_pca = trial.suggest_categorical("use_pca", [True, False])
        if use_pca:
            max_comp = min(n_fp, 500)
            n_comp = trial.suggest_int("pca_n_components", 10, max(10, max_comp))
            fp_transform = Pipeline([("pca", PCA(n_components=n_comp, svd_solver="randomized", random_state=42))])
        else:
            fp_transform = "passthrough"

        preprocessor = ColumnTransformer([
            ("desc", StandardScaler(), descriptors),
            ("fp", fp_transform, fingerprints)
        ])

        # MLP hyperparams
        n_layers = trial.suggest_int("n_layers", 1, 3)
        hidden_sizes = []
        for i in range(n_layers):
            hidden_sizes.append(trial.suggest_int(f"n_units_l{i}", 32, 512))
        hidden_tuple = tuple(hidden_sizes)
        activation = trial.suggest_categorical("activation", ["relu", "tanh"])
        alpha = trial.suggest_loguniform("alpha", 1e-6, 1e-1)
        learning_rate_init = trial.suggest_loguniform("learning_rate_init", 1e-5, 1e-1)
        max_iter = 2000

        mlp = MLPRegressor(hidden_layer_sizes=hidden_tuple, activation=activation,
                           alpha=alpha, learning_rate_init=learning_rate_init,
                           max_iter=max_iter, random_state=42)
        pipe = Pipeline([("preproc", preprocessor), ("mlp", mlp)])
        scores = cross_val_score(pipe, X, y, cv=cv_outer, scoring="r2", n_jobs=-1)
        return float(np.mean(scores))

    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(),
                                pruner=optuna.pruners.MedianPruner())
    study.optimize(objective, n_trials=n_trials, n_jobs=1, show_progress_bar=True)

    best = study.best_params
    if best.get("use_pca", False):
        fp_transform = Pipeline([("pca", PCA(n_components=best["pca_n_components"], random_state=42))])
    else:
        fp_transform = "passthrough"
    preprocessor_best = ColumnTransformer([
        ("desc", StandardScaler(), descriptors),
        ("fp", fp_transform, fingerprints)
    ])

    n_layers = best["n_layers"]
    hidden_sizes = tuple(best[f"n_units_l{i}"] for i in range(n_layers))
    best_mlp = MLPRegressor(hidden_layer_sizes=hidden_sizes,
                            activation=best["activation"],
                            alpha=best["alpha"],
                            learning_rate_init=best["learning_rate_init"],
                            max_iter=2000,
                            random_state=42)
    best_pipe = Pipeline([("preproc", preprocessor_best), ("mlp", best_mlp)])
    best_pipe.fit(X, y)
    joblib.dump(best_pipe, "mlp_optuna_pipeline.joblib")
    print("MLP best R2:", study.best_value)
    print("MLP best params:", study.best_params)
    return study, best_pipe

In [43]:
# -------------------- Random Forest --------------------------------
def run_optuna_rf(n_trials=50):
    def objective(trial):
        preprocessor = ColumnTransformer([
            ("desc", StandardScaler(), descriptors),
            ("fp", "passthrough", fingerprints)
        ])

        n_estimators = trial.suggest_int("n_estimators", 100, 500)
        max_depth = trial.suggest_int("max_depth", 3, 50)
        min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
        min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 20)
        max_features = trial.suggest_categorical("max_features", ["sqrt", "log2", 0.3, 0.5, 0.8])

        rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth,
                                   min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf,
                                   max_features=max_features, n_jobs=-1, random_state=42)
        pipe = Pipeline([("preproc", preprocessor), ("rf", rf)])
        scores = cross_val_score(pipe, X, y, cv=cv_outer, scoring="r2", n_jobs=-1)
        return float(np.mean(scores))

    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(),
                                pruner=optuna.pruners.MedianPruner())
    study.optimize(objective, n_trials=n_trials, n_jobs=1, show_progress_bar=True)

    best = study.best_params
    preprocessor_best = ColumnTransformer([
        ("desc", StandardScaler(), descriptors),
        ("fp", "passthrough", fingerprints)
    ])
    best_rf = RandomForestRegressor(n_estimators=best["n_estimators"], max_depth=best["max_depth"],
                                    min_samples_split=best["min_samples_split"],
                                    min_samples_leaf=best["min_samples_leaf"],
                                    max_features=best["max_features"], n_jobs=-1, random_state=42)
    best_pipe = Pipeline([("preproc", preprocessor_best), ("rf", best_rf)])
    best_pipe.fit(X, y)
    joblib.dump(best_pipe, "rf_optuna_pipeline.joblib")
    print("RF best R2:", study.best_value)
    print("RF best params:", study.best_params)
    return study, best_pipe

In [44]:
# ------------------------ XGBoost ---------------------------------
def run_optuna_xgb(n_trials=80, use_gpu=False):
    # Set tree_method to gpu_hist if use_gpu True (ensure XGBoost GPU support)
    def objective(trial):
        preprocessor = ColumnTransformer([
            ("desc", StandardScaler(), descriptors),
            ("fp", "passthrough", fingerprints)
        ])

        param = {
            "booster": trial.suggest_categorical("booster", ["gbtree", "dart"]),
            "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-8, 10.0),
            "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-8, 10.0),
            "subsample": trial.suggest_float("subsample", 0.4, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
            "learning_rate": trial.suggest_loguniform("learning_rate", 1e-4, 0.3),
            "max_depth": trial.suggest_int("max_depth", 3, 12),
            "n_estimators": trial.suggest_int("n_estimators", 50, 1000),
            "min_child_weight": trial.suggest_int("min_child_weight", 1, 20),
            "gamma": trial.suggest_loguniform("gamma", 1e-8, 10.0)
        }
        if use_gpu:
            param["tree_method"] = "gpu_hist"

        model = XGBRegressor(objective="reg:squarederror", random_state=42, n_jobs=-1, **param)
        pipe = Pipeline([("preproc", preprocessor), ("xgb", model)])
        scores = cross_val_score(pipe, X, y, cv=cv_outer, scoring="r2", n_jobs=-1)
        return float(np.mean(scores))

    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(),
                                pruner=optuna.pruners.MedianPruner())
    study.optimize(objective, n_trials=n_trials, n_jobs=1, show_progress_bar=True)

    best = study.best_params
    preprocessor_best = ColumnTransformer([
        ("desc", StandardScaler(), descriptors),
        ("fp", "passthrough", fingerprints)
    ])

    # reconstruct best xgb with best params
    if use_gpu:
        best["tree_method"] = "gpu_hist"
    best_xgb = XGBRegressor(objective="reg:squarederror", random_state=42, n_jobs=-1, **best)
    best_pipe = Pipeline([("preproc", preprocessor_best), ("xgb", best_xgb)])
    best_pipe.fit(X, y)
    joblib.dump(best_pipe, "xgb_optuna_pipeline.joblib")
    print("XGB best R2:", study.best_value)
    print("XGB best params:", study.best_params)
    return study, best_pipe

In [45]:
# ----------------------------
# Example: run all studies sequentially (adjust n_trials)
# ----------------------------
svm_study, svm_pipe = run_optuna_svm(n_trials=50)

[I 2025-10-05 18:08:04,948] A new study created in memory with name: no-name-9dc00423-79e2-42d9-bca1-4f6df5322fe7


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-05 18:08:08,645] Trial 0 finished with value: 0.7459018601518725 and parameters: {'use_pca': False, 'kernel': 'sigmoid', 'C': 9.767424132314616, 'epsilon': 0.00023550409490032, 'gamma_choice': 'auto'}. Best is trial 0 with value: 0.7459018601518725.
[I 2025-10-05 18:08:09,712] Trial 1 finished with value: 0.5425940184523494 and parameters: {'use_pca': True, 'pca_n_components': 77, 'kernel': 'sigmoid', 'C': 0.8204348004463511, 'epsilon': 0.14198729940196234, 'gamma_choice': 'scale'}. Best is trial 0 with value: 0.7459018601518725.
[I 2025-10-05 18:08:11,030] Trial 2 finished with value: 0.38022417441862866 and parameters: {'use_pca': False, 'kernel': 'rbf', 'C': 0.04476722029426198, 'epsilon': 0.1533295935051409, 'gamma_choice': 'scale'}. Best is trial 0 with value: 0.7459018601518725.
[I 2025-10-05 18:08:12,080] Trial 3 finished with value: 0.6748075226779413 and parameters: {'use_pca': False, 'kernel': 'rbf', 'C': 0.23968570301494116, 'epsilon': 0.00105892135282442, 'gamma_

In [46]:
knn_study, knn_pipe = run_optuna_knn(n_trials=50)

[I 2025-10-05 18:09:10,660] A new study created in memory with name: no-name-a056d05b-fe71-491e-9250-e047b8aae676


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-05 18:09:11,295] Trial 0 finished with value: 0.7488824738800499 and parameters: {'use_pca': False, 'n_neighbors': 12, 'weights': 'uniform', 'p': 1}. Best is trial 0 with value: 0.7488824738800499.
[I 2025-10-05 18:09:11,794] Trial 1 finished with value: 0.6918954337532598 and parameters: {'use_pca': True, 'pca_n_components': 63, 'n_neighbors': 23, 'weights': 'uniform', 'p': 2}. Best is trial 0 with value: 0.7488824738800499.
[I 2025-10-05 18:09:12,056] Trial 2 finished with value: 0.7573195145761333 and parameters: {'use_pca': False, 'n_neighbors': 6, 'weights': 'uniform', 'p': 2}. Best is trial 2 with value: 0.7573195145761333.
[I 2025-10-05 18:09:12,613] Trial 3 finished with value: 0.7378017721356777 and parameters: {'use_pca': False, 'n_neighbors': 3, 'weights': 'distance', 'p': 1}. Best is trial 2 with value: 0.7573195145761333.
[I 2025-10-05 18:09:13,482] Trial 4 finished with value: 0.7415701889967015 and parameters: {'use_pca': True, 'pca_n_components': 192, 'n_neig

In [47]:
rf_study, rf_pipe = run_optuna_rf(n_trials=50)

[I 2025-10-05 18:09:35,371] A new study created in memory with name: no-name-6da7abf9-cc23-4c29-9b55-bf148460b74c


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-10-05 18:09:38,186] Trial 0 finished with value: 0.5827287825885963 and parameters: {'n_estimators': 277, 'max_depth': 25, 'min_samples_split': 12, 'min_samples_leaf': 14, 'max_features': 'log2'}. Best is trial 0 with value: 0.5827287825885963.
[I 2025-10-05 18:09:42,495] Trial 1 finished with value: 0.6596017160633199 and parameters: {'n_estimators': 405, 'max_depth': 46, 'min_samples_split': 19, 'min_samples_leaf': 14, 'max_features': 'sqrt'}. Best is trial 1 with value: 0.6596017160633199.
[I 2025-10-05 18:09:48,219] Trial 2 finished with value: 0.7182379508568626 and parameters: {'n_estimators': 238, 'max_depth': 7, 'min_samples_split': 13, 'min_samples_leaf': 13, 'max_features': 0.3}. Best is trial 2 with value: 0.7182379508568626.
[I 2025-10-05 18:09:58,513] Trial 3 finished with value: 0.7653231166879009 and parameters: {'n_estimators': 337, 'max_depth': 24, 'min_samples_split': 14, 'min_samples_leaf': 4, 'max_features': 0.3}. Best is trial 3 with value: 0.76532311668790

# 7. Try a meta model or ensembler of the best models

In [48]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_predict

# reproducible folds
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# 1) Get OOF predictions for each base model (these preds are made by models
#    that were trained without the corresponding sample — no leakage)
print("Generating OOF preds (this may take time)...")
oof_knn = cross_val_predict(knn_pipe, X, y, cv=kf, n_jobs=-1, method='predict')
oof_svm = cross_val_predict(svm_pipe, X, y, cv=kf, n_jobs=-1, method='predict')
oof_rf  = cross_val_predict(rf_pipe,  X, y, cv=kf, n_jobs=-1, method='predict')

# Stack OOF predictions (n_samples x n_models)
stack_oof = np.vstack([oof_knn, oof_svm, oof_rf]).T

# 2) Simple average ensemble
ens_mean = stack_oof.mean(axis=1)
r2_mean = r2_score(y, ens_mean)
rmse_mean = np.sqrt(mean_squared_error(y, ens_mean))
print(f"Simple average ensemble -> R2: {r2_mean:.4f}, RMSE: {rmse_mean:.4f}")

Generating OOF preds (this may take time)...
Simple average ensemble -> R2: 0.7883, RMSE: 0.8263


In [49]:
# 3) Stacking: train a Ridge meta-learner on the OOF stack
meta = Ridge(alpha=1.0)
meta.fit(stack_oof, y)                 # training on OOF preds is OK (no leakage)
ens_stack = meta.predict(stack_oof)    # predictions on the same OOF matrix
r2_stack = r2_score(y, ens_stack)
rmse_stack = np.sqrt(mean_squared_error(y, ens_stack))
print(f"Stacking (Ridge) ensemble -> R2: {r2_stack:.4f}, RMSE: {rmse_stack:.4f}")

# Show meta-learner weights
print("Meta-learner coefficients (weights):", meta.coef_)
print("Meta-learner intercept:", meta.intercept_)

Stacking (Ridge) ensemble -> R2: 0.7920, RMSE: 0.8190
Meta-learner coefficients (weights): [0.3405945  0.66396643 0.04286632]
Meta-learner intercept: -0.32256528491550185


In [50]:
joblib.dump(meta, "ensemble_meta_ridge.joblib")
print("Saved final base pipelines and meta-learner.")

Saved final base pipelines and meta-learner.
